## MNIST Digit Classification using a CNN 

In [7]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow import keras
import cv2
import numpy as np
import matplotlib.pyplot as plt


In [4]:
mnist = tf.keras.datasets.mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0

In [3]:
img_rows , img_cols = 28, 28

In [8]:
if keras.backend.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [9]:
num_category = 10
y_train = keras.utils.to_categorical(y_train, num_category)
y_test = keras.utils.to_categorical(y_test, num_category)
y_train[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [13]:
model = keras.Sequential()
#convolutional layer with rectified linear unit activation
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
#32 convolution filters used each of size 3x3
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
#64 convolution filters used each of size 3x3
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
#add dropout
model.add(keras.layers.Dropout(0.25))
#flatten
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
#dropout
model.add(keras.layers.Dropout(0.5))
#softmax to output probabilities
model.add(keras.layers.Dense(num_category, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)              

In [17]:
num_epoch = 10
batch_size = 128
model_log = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=num_epoch,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/10
469/469 [==============================] - 138s 294ms/step - loss: 0.3688 - accuracy: 0.8867 - val_loss: 0.1765 - val_accuracy: 0.9464
Epoch 2/10
469/469 [==============================] - 138s 293ms/step - loss: 0.3583 - accuracy: 0.8882 - val_loss: 0.1656 - val_accuracy: 0.9506
Epoch 3/10
469/469 [==============================] - 142s 304ms/step - loss: 0.3486 - accuracy: 0.8917 - val_loss: 0.1628 - val_accuracy: 0.9499
Epoch 4/10
469/469 [==============================] - 138s 295ms/step - loss: 0.3407 - accuracy: 0.8952 - val_loss: 0.1601 - val_accuracy: 0.9530
Epoch 5/10
469/469 [==============================] - 138s 295ms/step - loss: 0.3287 - accuracy: 0.8980 - val_loss: 0.1509 - val_accuracy: 0.9575
Epoch 6/10
469/469 [==============================] - 139s 296ms/step - loss: 0.3222 - accuracy: 0.9004 - val_loss: 0.1606 - val_accuracy: 0.9516
Epoch 7/10
469/469 [==============================] - 138s 295ms/step - loss: 0.3139 - accuracy: 0.9035 - val_loss: 0.1416 -

In [18]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.13200989365577698
Test accuracy: 0.9621999859809875
